In [ ]:
import numpy as np
import torch
from vectorhash_imported import *
from vectorhash_convered import *
from nd_scaffold import GridScaffold as GS
import math
from scipy.stats import norm
from matrix_initializers import ConstantInitializer, SparseMatrixByScalingInitializer



lambdas = [3, 4, 5]
Ng = sum([i**2 for i in lambdas])


percent_nonzero_relu=0.01
W_gh_var=1.0
sparse_initialization=0.1
T=0.01
W_hg_std = math.sqrt(W_gh_var)
W_hg_mean = (
    -W_hg_std * norm.ppf(1 - percent_nonzero_relu) / math.sqrt(len(lambdas))
)
h_normal_mean = len(lambdas) * W_hg_mean
h_normal_std = math.sqrt(len(lambdas)) * W_hg_std
relu_theta = 0

# SparseMatrixByScalingInitializer(mean=W_hg_mean, scale=W_hg_std, device="cpu")

gs = GS(
    shapes= [(3, 3), (4, 4), (5, 5)],
    N_h= 400,
    input_size= 2000,
    h_normal_mean=h_normal_mean,
    h_normal_std=h_normal_std,
    device=None,
    sparse_matrix_initializer=SparseMatrixBySparsityInitializer(sparsity=0.6, device="cpu"),
    relu_theta=0.5,
    from_checkpoint=False,
    T=1,
    continualupdate=True,
    ratshift=False,
    initialize_W_gh_with_zeroes=False,
    pseudo_inverse=False,
    batch_update=False,
    use_h_fix=False,
    learned_pseudo=True,
    epsilon=0.01,
    calculate_update_scaling_method="norm",
    MagicMath=False,
    sanity_check=False,
    calculate_g_method="hairpin"
)






In [ ]:
print(gs.G[0])

In [3]:
from numpy.random import randn

lambdas = [3,4,5]                   # module period
M = len(lambdas)                             # num modules
Ng = np.sum(np.square(lambdas))                             # num grid cells
Npos = np.prod(lambdas)   
Npos = Npos*Npos
gbook = gs.G
Ns = 2000#84*84*3                           # num of sensory cells set at Npos, can be larger
Np_lst = [400] #np.arange(25, 425, 25)     # num place cells
pflip = 0.0                         #measure of noise injected in s (prob of flipping if binary, gaussian noise if cts)
Niter = 1                           # number of iterations for scaffold dynamics
nruns=1
sparsity=0        #Dummy param for older code, not used currently
sbook = np.sign(randn(Ns, Npos))
Npatts_lst = np.arange(1,Ns+1,200)  # number of patterns to train on 

# Npatts is 1, 201, 401 etc... 3401

In [ ]:
##Choose experiment to run

#Base case
err_pc, err_gc, err_sens, err_senscup, err_sensl1, ga, gd, gt = capacity(senstrans_gs_vectorized_patts, lambdas, Ng, Np_lst, pflip, Niter, Npos, 
                                          gbook, Npatts_lst, nruns, Ns, sbook, sparsity, noise_level="none", grid_scaffold=gs, W_hg_mean=W_hg_mean, W_hg_std=W_hg_std)

#Place states chosen to be random vectors with same sparsity as base case (teal curves in Fig. 3)
#err_pc, err_gc, err_sens, err_senscup, err_sensl1 = capacity(senstrans_gs_random_sparse_p, lambdas, Ng, Np_lst, pflip, Niter, Npos, 
                                          #gbook, Npatts_lst, nruns, Ns, sbook, sparsity)
#Assuming linear hippocampal activations
# err_pc, err_gc, err_sens, err_senscup, err_sensl1 = capacity(senstrans_gs_linear_p, lambdas, Ng, Np_lst, pflip, Niter, Npos, 
                                          # gbook, Npatts_lst, nruns, Ns, sbook, sparsity)
                                          
#Use gbook as a spiraling outward + linear activation (for SI Fig. S13)
# err_pc, err_gc, err_sens, err_senscup, err_sensl1 = capacity(senstrans_gs_linear_p_spiral, lambdas, Ng, Np_lst, pflip, Niter, Npos, 
                                          # gbook, Npatts_lst, nruns, Ns, sbook, sparsity)                                          

In [ ]:
print(err_pc)
print(err_gc)
print(err_sens)
print(err_senscup)
print(err_sensl1)

In [ ]:
normlizd_l1 = err_sensl1
Npatts = np.array(nruns*[Npatts_lst])   # Npatts_lst repeated nruns times
Npatts = Npatts.T
print(Npatts.shape)

m = 1 - (2*normlizd_l1) 

a = (1+m)/2
b = (1-m)/2

print(a)
print(b)
a1 = [[[1.0        ],[0.60587562],[0.57078304],[0.55598336],[0.54750687],[0.541999  ],[0.53836345],[0.53536188],[0.53285103],[0.53089478]]]
b1 = [[[0.0      ],[0.39412438],[0.42921696],[0.44401664],[0.45249313],[0.458001  ],[0.46163655],[0.46463812],[0.46714897],[0.46910522]]]
a1 = torch.tensor(a1)
b1 = torch.tensor(b1)
a = torch.abs(torch.tensor(a))
b = torch.abs(torch.tensor(b))

S = - a * np.log2(a) - b * np.log2(b)
S = np.where(m==1, np.zeros_like(S), S)

S1 = - a1 * np.log2(a1) - b1 * np.log2(b1)
S1 = np.where(m==1, np.zeros_like(S1), S1)


MI = 1 - S
MI1 = 1 - S1
plt.errorbar(Npatts_lst,MI[0].mean(axis=1),yerr=MI[0].std(axis=1),lw=2); plt.xscale('log'); plt.yscale('log')
plt.errorbar(Npatts_lst,MI1[0].mean(axis=1),yerr=MI1[0].std(axis=1),lw=2); plt.xscale('log'); plt.yscale('log')
plt.legend(['US', 'VECTORHASH'])
plt.title('MI per inp bit vs num patts')

plt.xlim(xmin=100)
plt.ylabel('MI per inp bit')
plt.xlabel('num patts')
plt.show()

In [ ]:
###Baselines
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm


def cap(W,bound):
    W1=torch.where(W>bound,bound*torch.ones(W.shape),W)
    W2=torch.where(W1<-bound,-bound*torch.ones(W.shape),W1)
    return W2

def corrupt_p(codebook,p=0.1,booktype='-11'):
    rand_indices = torch.sign(torch.random.uniform(size=codebook.shape)- p )
    if booktype=='-11':
        return torch.multiply(codebook,rand_indices)
    elif booktype=='01':
        return abs(codebook - 0.5*(-rand_indices+1))
    elif booktype=='cts':
        return codebook + torch.random.normal(0,1,size=codebook.shape)*p
    else:
        print("codebook should be -11; 01; or cts")
        return 0


def get_weights(patterns,connectivity):
    if connectivity is 'standard':
        if learning == 'hebbian':
            W = patts @ patts.T
        elif learning == 'sparsehebbian':
            prob = sparsity #np.sum(patts)/patts.shape[0]/patts.shape[1]
            W =(1/patts.shape[0])* (patts - prob) @ (patts.T - prob)
        elif learning == 'pinv':
            W= patts @ np.linalg.pinv(patts)
        elif learning == 'bounded_hebbian':
            num_patts = patts.shape[1]
            num_nodes = patts.shape[0]
            W = np.zeros((num_nodes,num_nodes))
            for i in range(num_patts):
                Wtmp = np.outer(patts[:,i] , patts[:,i])/np.sqrt(num_nodes)
                # ~ print(np.amax(Wtmp))
                W = cap(Wtmp + W,bound)
        W = W - torch.diag(torch.diag(W))
    else:
        N = connectivity.shape[0]
        W = sparse.lil_matrix(connectivity.shape)
        for i in range(N):
            for j in connectivity.rows[i]:
                W[i,j] = np.dot(patterns[i],patterns[j])
        W.setdiag(0)
    return W


def entropy(inlist):
    ent = np.zeros(len(inlist))
    for idx,x in enumerate(inlist):
        if x == 0 or x == 1:
            ent[idx] = 0
        else:
            ent[idx] = -1 * ( x*np.log2(x) + (1-x)*np.log2(1-x) )
    return ent

In [ ]:
nruns=1
iterations=100
N = 708
corrupt_fraction = 0.0
Npatts_list = np.arange(1,800,10)
connectivity='standard' # Standard fully connected Hopfield network. For sparse connectivity use the next cell
# learning can be 'hebbian', 'bounded_hebbian', 'pinv', or 'sparsehebbian' for sparse hopfield network
learning='bounded_hebbian'
bound=0.3  #Use bound param if learning='bounded_hebbian'

init_overlap = torch.zeros((nruns,*Npatts_list.shape))
final_overlap = torch.zeros((nruns,*Npatts_list.shape))
MI_hc = torch.zeros((nruns,*Npatts_list.shape))


for runidx in range(nruns):
    print("runidx = "+str(runidx))
    
    if learning == 'sparsehebbian':
        # sparse hopfiled 0/1 code
        sparsity = 0.2
        patterns = 1*(torch.random.rand(N,Npatts_list.max()) > (1-sparsity))
    else:
        patterns = torch.sign(torch.random.normal(0,1,(N,Npatts_list.max())))

    
    for idx,Npatts in enumerate(tqdm(Npatts_list)):
        #print(Npatts)
        patts = patterns[:,:Npatts]
        cor_patts = patterns[:,:Npatts]
        W = get_weights(patts,connectivity)
        
        if learning == 'sparsehebbian':
            # sparse hopfield
            theta = torch.sum(W-torch.diag(W), axis=1)
            theta=0.05 #0.04 #0
            rep = (torch.sign(W@cor_patts - theta)+1)/2            
        else:
            rep = torch.sign(W@cor_patts)

        init_overlap[runidx,idx] = np.average(np.einsum('ij,ij->j',rep,patts)/N) 

        rep1 = np.copy(rep)
        for ite in range(iterations-1):
            if learning == 'sparsehebbian':
                rep = (np.sign(W@rep - theta)+1)/2
            else:
                rep = np.sign(W@rep)
            
            if np.sum(abs(rep - rep1))>0:
                rep1 = np.copy(rep)
            else:
                # print("converged at "+str(ite))
                break
        err = np.einsum('ij,ij->j',rep,patts)/N
        overlap = np.average(err) 
        final_overlap[runidx,idx] = overlap #err
        
        if learning=='sparsehebbian':
            q = np.sum(np.abs(rep), axis=0) / N  # sparse hopfield
            m = err
            p = np.sum(patts, axis=0)/patts.shape[0]
            P1e = 1 - (m/p)
            P0e = (q-m)/(1-p)
            MI_hc[runidx,idx] =  np.average( entropy(q) - ( p*entropy(P1e) + (1-p)*entropy(P0e) ) )


# print(init_overlap)
# print(final_overlap)

results_dir = "continuum_results"
# filename = f"sparseconnhopfield__mutualinfo_N={N}_noise={corrupt_fraction}_gamma={gamma}_iter={iterations}_nruns={nruns}"
filename = f"stdhopfield__mutualinfo_N={N}_noise={corrupt_fraction}_iter={iterations}_nruns={nruns}"
# filename = f"pinvhopfield__mutualinfo_N={N}_noise={corrupt_fraction}_iter={iterations}_nruns={nruns}"
# filename = f"sparsehopfield__mutualinfo_N={N}_noise={corrupt_fraction}_p={sparsity}_iter={iterations}_nruns={nruns}"
# filename = f"boundedhopfield__mutualinfo_N={N}_noise={corrupt_fraction}_bound={bound}_iter={iterations}_nruns={nruns}"


fig1 = plt.figure(1)
plt.plot(Npatts_list,init_overlap.mean(axis=0), label='single, corrupt='+str(corrupt_fraction));
plt.plot(Npatts_list,final_overlap.mean(axis=0), label='final, corrupt='+str(corrupt_fraction));
plt.legend()
plt.xlabel('Number of patterns')
plt.ylabel("Overlap");
plt.title(r"N = "+str(N)+", $W$");
plt.show()
# exit()
# fig1.savefig(f"{results_dir}/Overlap_{filename}.png")

if learning=='sparsehebbian':
    print("MI already calculated in loop")
else:
    m = final_overlap
    a = (1+m)/2
    b = (1-m)/2

    S = - a * np.log2(a) - b * np.log2(b)
    S = np.where(m==1, np.zeros_like(S), S)

    MI_hc = 1 - S


fig2 = plt.figure(1)
plt.errorbar(Npatts_list,MI_hc.mean(axis=0),yerr=MI_hc.std(axis=0), label='final, corrupt='+str(corrupt_fraction)); #plt.xscale('log'); plt.yscale('log');
plt.legend()
plt.xlabel('Number of patterns')
plt.ylabel("MI");
plt.title(r"N = "+str(N)+", $W$");
plt.show()
# fig2.savefig(f"{results_dir}/MI_{filename}.png")

data = {
    "N": N,
    "init_overlap": init_overlap,
    "m": final_overlap,
    "MI": MI_hc,
    "Npatts_list": Npatts_list,
    "noise": corrupt_fraction,
    # "q": q  #needed for sparse hebbian
    # "bound": bound #needed for bounded hopfield
}
# write_pkl(f"{results_dir}/{filename}", data)